# Homework 5

Estimate the distribution of log luminance levels in the Ruderman natural image dataset. For each image, transform it by taking the log
of the pixel luminance values, and subtracting the mean value of log luminance across each image. Show the estimate of the error bars on this distribution of mean-subtracted log-luminances, y, by thinking about what “independent samples” are in this dataset (you do not need to do a full bootstrap analysis). Show that the left- and right- tails of this distribution are approximately exponential, i.e., P + (y) / exp( μ + |y|) and P (y) / exp( μ |y|), where P ± stand for the positive (or, respectively, the negative) tails of the log-luminance distribution. Do this by finding the two best-fitting constants, μ + and μ and plotting the tails on top of the normalized distribution for y. A good choice to define the positive and negative tails is to use the threshold of |y| > 1. How you do the fitting (linear regression in log-luminance space, 2 fitting of raw histograms) is up to you, but explain clearly what you did.

estimate the pdf on each image: error is std on ??.
same binning is crucial!